In [ ]:
import requests

redmine_url = "https://new-redmine5.senseitech.com"
api_key = "9dc4f57007d2f560ae091dfd861ad711070dbb25"

# Headers for authentication
headers = {
    'X-Redmine-API-Key': api_key,
    'Content-Type': 'application/json'
}

# Function to fetch all issues with pagination
def get_all_issues():
    issues = []
    limit = 100  # Number of issues per page (max: 100)
    offset = 0   # Start from the first issue

    while True:
        issues_url = f"{redmine_url}/issues.json?limit={limit}&offset={offset}"
        response = requests.get(issues_url, headers=headers)
        
        if response.status_code == 200:
            data = response.json()
            current_issues = data.get('issues', [])
            
            # If there are no more issues, break the loop
            if not current_issues:
                break
            
            # Add fetched issues to the list
            issues.extend(current_issues)
            offset += limit
        else:
            print(f"Failed to fetch issues. Status Code: {response.status_code}")
            break

    return issues

# Function to fetch a specific issue by issue_id
def get_issue(issue_id):
    issue_url = f"{redmine_url}/issues/{issue_id}.json"
    response = requests.get(issue_url, headers=headers)
    
    if response.status_code == 200:
        issue_details = response.json().get('issue', {})
        return issue_details
    else:
        print(f"Failed to fetch issue details for ID {issue_id}. Status Code: {response.status_code}")
        return None

# Fetch all issues
all_issues = get_all_issues()

if all_issues:
    for issue in all_issues:
        issue_id = issue['id']  # Dynamically set the issue_id
        issue_details = get_issue(issue_id)
        
        if issue_details:
            print(f"Issue ID: {issue_details['id']}")
            print(f"Subject: {issue_details['subject']}")
            print(f"Description: {issue_details['description']}")
            print(f"Status: {issue_details['status']['name']}")
            print(f"Priority: {issue_details['priority']['name']}")
            print(f"Assigned to: {issue_details.get('assigned_to', {}).get('name', 'Not assigned')}")
        print("\n" + "-"*50 + "\n")

In [7]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from transformers import pipeline

# Download stopwords from NLTK (if not already done)
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load the CSV file with ticket descriptions
df = pd.read_csv('redmine_issues.csv')

# Initialize the emotion analysis pipeline using a pre-trained model for emotion classification
emotion_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

# Define a mapping to align model output with your target emotions
emotion_mapping = {
    'anger': 'Anger',
    'sadness': 'Disappointment',
    'fear': 'Confusion',
    'joy': 'Satisfaction',
    'neutral': 'Neutral',
    'disgust': 'Frustration'
}

# Function to preprocess the text
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters, numbers, and punctuation
    text = re.sub(r'[^a-z\s]', '', text)
    
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    return text

# Function to detect emotion in a description
def detect_emotion(text):
    # Preprocess the text
    preprocessed_text = preprocess_text(text)
    
    # Truncate text if it's too long for the model (max 512 tokens)
    truncated_text = preprocessed_text[:512]
    
    # Get emotion predictions
    predictions = emotion_classifier(truncated_text)
    
    # Extract the top emotion with the highest score
    top_emotion = max(predictions[0], key=lambda x: x['score'])['label']
    
    # Map the detected emotion to the specified categories
    return emotion_mapping.get(top_emotion, 'Neutral')

# Apply emotion detection to each ticket description
df['Detected Emotion'] = df['Description'].apply(lambda text: detect_emotion(str(text)))

# Save the results to a new CSV file
df[['Description', 'Detected Emotion']].to_csv('issue_emotions_detected.csv', index=False)

print("Emotion detection completed and saved to issue_emotions_detected.csv")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ssure\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\ssure\Desktop\Ticket_management\env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\Users\ssure\Desktop\Ticket_management\env\Lib\site-packages\transformers\pipelines\text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Emotion detection completed and saved to issue_emotions_detected.csv


AFTER PREPROCESSING

In [9]:
import requests
import pandas as pd

redmine_url = "https://new-redmine5.senseitech.com"
api_key = "9dc4f57007d2f560ae091dfd861ad711070dbb25"

# Headers for authentication
headers = {
    'X-Redmine-API-Key': api_key,
    'Content-Type': 'application/json'
}

# Function to fetch all issues with pagination
def get_all_issues():
    issues = []
    limit = 100  # Number of issues per page (max: 100)
    offset = 0   # Start from the first issue

    while True:
        issues_url = f"{redmine_url}/issues.json?limit={limit}&offset={offset}"
        response = requests.get(issues_url, headers=headers)
        
        if response.status_code == 200:
            data = response.json()
            current_issues = data.get('issues', [])
            
            # If there are no more issues, break the loop
            if not current_issues:
                break
            
            # Add fetched issues to the list
            issues.extend(current_issues)
            offset += limit
        else:
            print(f"Failed to fetch issues. Status Code: {response.status_code}")
            break

    return issues

# Function to fetch a specific issue by issue_id
def get_issue(issue_id):
    issue_url = f"{redmine_url}/issues/{issue_id}.json"
    response = requests.get(issue_url, headers=headers)
    
    if response.status_code == 200:
        issue_details = response.json().get('issue', {})
        return issue_details
    else:
        print(f"Failed to fetch issue details for ID {issue_id}. Status Code: {response.status_code}")
        return None

# Fetch all issues
all_issues = get_all_issues()

# Prepare data for DataFrame
issue_data = []
if all_issues:
    for issue in all_issues:
        issue_id = issue['id']  # Dynamically set the issue_id
        issue_details = get_issue(issue_id)
        
        if issue_details:
            issue_info = {
                "Issue ID": issue_details['id'],
                "Subject": issue_details['subject'],
                "Description": issue_details.get('description', '').replace('\n', ' '),  # Replace newlines with spaces
                "Status": issue_details['status']['name'],
                "Priority": issue_details['priority']['name'] if 'priority' in issue_details else 'Not specified',
                "Assigned To": issue_details.get('assigned_to', {}).get('name', 'Not assigned')
            }
            issue_data.append(issue_info)

# Create a DataFrame from the collected data
df_issues = pd.DataFrame(issue_data)

# Save DataFrame to a CSV file
df_issues.to_csv('redmine_issues.csv', index=False)

print("Issues have been successfully saved to redmine_issues.csv.")

Issues have been successfully saved to redmine_issues.csv.


WITHOUT PREPROCESSING



In [29]:
import pandas as pd
from transformers import pipeline

# Load the CSV file with ticket descriptions
df = pd.read_csv('redmine_issues.csv')

# Initialize the emotion analysis pipeline using a pre-trained model for emotion classification
emotion_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

# Define a mapping to align model output with your target emotions
emotion_mapping = {
    'anger': 'Anger',
    'sadness': 'Disappointment',
    'fear': 'Confusion',
    'joy': 'Satisfaction',
    'neutral': 'Neutral',
    'disgust': 'Frustration'
}

# Function to detect emotion in a description
def detect_emotion(text):
    # Truncate text if it's too long for the model (max 512 tokens)
    truncated_text = text[:512]
    
    # Get emotion predictions
    predictions = emotion_classifier(truncated_text)
    
    # Extract the top emotion with the highest score
    top_emotion = max(predictions[0], key=lambda x: x['score'])['label']
    
    # Map the detected emotion to the specified categories
    return emotion_mapping.get(top_emotion, 'Neutral')

# Apply emotion detection to each ticket description
df['Detected Emotion'] = df['Description'].apply(lambda text: detect_emotion(str(text)))

# Save the results to a new CSV file
df[['Description', 'Detected Emotion']].to_csv('issue_emotions_detected.csv', index=False)

print("Emotion detection completed and saved to issue_emotions_detected.csv")


C:\Users\ssure\Desktop\Ticket_management\env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\Users\ssure\Desktop\Ticket_management\env\Lib\site-packages\transformers\pipelines\text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Emotion detection completed and saved to issue_emotions_detected.csv
